In [ ]:
!pip install -r requirements.txt

In [1]:
# Import
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../raw_data/export_2024-03-20_cleaned.csv')
data.head()

,Unnamed: 0,source,theme,soustheme,title,description,link,published,image,added_date
0,0,Ça commence aujourd'hui : youtube,culture,quotidien,Pornographie : quel chemin les a menées vers l...,Retrouvez les intégrales de l’émission sur Fra...,https://www.youtube.com/watch?v=kBtSb_mzokU,2024-03-19T18:00:22+00:00,https://www.youtube.com/v/kBtSb_mzokU?version=3,2024-03-20 09:00:00
1,1,Ça commence aujourd'hui : youtube,culture,quotidien,"Le parcours d'Emilie, alias Liza del Sierra, d...",Retrouvez les intégrales de l’émission sur Fra...,https://www.youtube.com/watch?v=doOukyUvrjk,2024-03-19T15:22:46+00:00,https://www.youtube.com/v/doOukyUvrjk?version=3,2024-03-20 09:00:00
2,2,Ça commence aujourd'hui : youtube,culture,quotidien,Maryline est créatrice de contenus pour adulte...,Retrouvez les intégrales de l’émission sur Fra...,https://www.youtube.com/watch?v=TdfnyZ_PUHA,2024-03-19T15:22:40+00:00,https://www.youtube.com/v/TdfnyZ_PUHA?version=3,2024-03-20 09:00:00
3,3,Ça commence aujourd'hui : youtube,culture,quotidien,"Travailleuse du sexe virtuelle : ""ça a aidé à ...",Retrouvez les intégrales de l’émission sur Fra...,https://www.youtube.com/watch?v=wHt8Qh-XWvQ,2024-03-19T15:22:35+00:00,https://www.youtube.com/v/wHt8Qh-XWvQ?version=3,2024-03-20 09:00:00
4,4,École nationale des Chartes : youtube,culture,NaN,"Marie Arconati Visconti, une vie à l’École des...","Conférence de Martine Poulain, conservatrice g...",https://www.youtube.com/watch?v=dMuWvA1J29Y,2024-03-19T09:24:59+00:00,https://www.youtube.com/v/dMuWvA1J29Y?version=3,2024-03-20 09:00:00


# Création de la TABLE Category

In [3]:
data.theme.unique()
theme_list = ['culture', 'musique', 'technologie', 'politique', 'sport', 'economie',
         'environnement', 'general', 'emploi', 'cite', 'management', 'presse',
         'nustrale','tourisme']
data = data[data.theme.isin(theme_list)]
data.theme.unique()

array(['culture', 'musique', 'technologie', 'politique', 'sport',
       'economie', 'environnement', 'general', 'cite', 'management',
       'presse', 'emploi', 'nustrale', 'tourisme'], dtype=object)

In [7]:
cat_df = pd.DataFrame({'category_id': list(range(1, len(theme_list) + 1)),
        'cat_name': theme_list})
cat_df

,category_id,cat_name
0,1,culture
1,2,musique
2,3,technologie
3,4,politique
4,5,sport
5,6,economie
6,7,environnement
7,8,general
8,9,emploi
9,10,cite


In [8]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Path to your service account key file
key_path = "../credentials/mathieu_credentials.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

In [9]:
# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
CAT_TABLE='category_dataset'
CAT_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{CAT_TABLE}' # the-mdr-project.live_mdr.category_dataset

In [13]:
client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)
write_mode = 'WRITE_TRUNCATE'
job_config = bigquery.LoadJobConfig(write_disposition=write_mode)
job = client.load_table_from_dataframe(cat_df, CAT_TABLE_ID, job_config=job_config)
result = job.result()

## Test d'une query sur la category

In [14]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Path to your service account key file
key_path = "mathieu_credentials.json"

# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
CAT_TABLE='category_dataset'
CAT_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{CAT_TABLE}' # the-mdr-project.live_mdr.category_dataset

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)
query = """SELECT *
            FROM the-mdr-project.live_mdr.category_dataset"""
query_job = client.query(query)
result = query_job.result()
df = result.to_dataframe()
df

,category_id,cat_name
0,1,culture
1,2,musique
2,3,technologie
3,4,politique
4,5,sport
5,6,economie
6,7,environnement
7,8,general
8,9,emploi
9,10,cite


# Implémentation de la TABLE news

In [10]:
# Dataframe which is associate withe the news table
result = data.merge(cat_df, left_on='theme', right_on='cat_name')
result['news_id'] = np.arange(1, len(result) + 1)
news_df = result[['news_id', 'category_id', 'title', 'description', 'link','image', 'added_date', 'source', 'soustheme']]
news_df.rename(columns={'soustheme': 'sub_cat'}, inplace=True)
news_df

/var/folders/7_/qgxv4n054fz23g05r4yrv3kw0000gn/T/ipykernel_38766/2971487670.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df.rename(columns={'soustheme': 'sub_cat'}, inplace=True)


,news_id,category_id,title,description,link,image,added_date,source,sub_cat
0,1,1,Pornographie : quel chemin les a menées vers l...,Retrouvez les intégrales de l’émission sur Fra...,https://www.youtube.com/watch?v=kBtSb_mzokU,https://www.youtube.com/v/kBtSb_mzokU?version=3,2024-03-20 09:00:00,Ça commence aujourd'hui : youtube,quotidien
1,2,1,"Le parcours d'Emilie, alias Liza del Sierra, d...",Retrouvez les intégrales de l’émission sur Fra...,https://www.youtube.com/watch?v=doOukyUvrjk,https://www.youtube.com/v/doOukyUvrjk?version=3,2024-03-20 09:00:00,Ça commence aujourd'hui : youtube,quotidien
2,3,1,Maryline est créatrice de contenus pour adulte...,Retrouvez les intégrales de l’émission sur Fra...,https://www.youtube.com/watch?v=TdfnyZ_PUHA,https://www.youtube.com/v/TdfnyZ_PUHA?version=3,2024-03-20 09:00:00,Ça commence aujourd'hui : youtube,quotidien
3,4,1,"Travailleuse du sexe virtuelle : ""ça a aidé à ...",Retrouvez les intégrales de l’émission sur Fra...,https://www.youtube.com/watch?v=wHt8Qh-XWvQ,https://www.youtube.com/v/wHt8Qh-XWvQ?version=3,2024-03-20 09:00:00,Ça commence aujourd'hui : youtube,quotidien
4,5,1,"Marie Arconati Visconti, une vie à l’École des...","Conférence de Martine Poulain, conservatrice g...",https://www.youtube.com/watch?v=dMuWvA1J29Y,https://www.youtube.com/v/dMuWvA1J29Y?version=3,2024-03-20 09:00:00,École nationale des Chartes : youtube,NaN
...,...,...,...,...,...,...,...,...,...
345937,345938,11,Comment mesurer le management ?,Certains considèrent que le management constit...,https://www.hbrfrance.fr/management/comment-me...,https://tra.img.pmdstatic.net/fit/~1~hbr~2024~...,2024-03-07 22:00:00,HBR,general
345938,345939,11,Les innovations managériales fondées sur le po...,Alors que la troisième révolution industrielle...,https://www.hbrfrance.fr/chroniques-experts/20...,https://tra.img.pmdstatic.net/fit/~1~hbr~2024~...,2024-03-07 22:00:00,HBR,general
345939,345940,11,Penser le futur du travail implique aussi de p...,L'IA et la crise écologique transforment le tr...,https://www.hbrfrance.fr/strategie/penser-le-f...,https://tra.img.pmdstatic.net/fit/~1~hbr~2024~...,2024-03-07 22:00:00,HBR,general
345940,345941,11,Comment notre passion pour les réseaux sociaux...,Dans un contexte où les utilisateurs produisen...,https://www.hbrfrance.fr/magazine/2020/12/3231...,https://tra.img.pmdstatic.net/fit/~1~hbr~2024~...,2024-03-07 22:00:00,HBR,general


In [11]:
# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
NEWS_TABLE='news_dataset'
NEWS_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{NEWS_TABLE}' # the-mdr-project.live_mdr.news_dataset

# Path to your service account key file
key_path = "../credentials/mathieu_credentials.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

In [12]:
client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)
write_mode = 'WRITE_TRUNCATE'
job_config = bigquery.LoadJobConfig(write_disposition=write_mode)
job = client.load_table_from_dataframe(news_df, NEWS_TABLE_ID, job_config=job_config)
result = job.result()

## Test de l'implementation avec une query

In [13]:
query = """SELECT *
            FROM the-mdr-project.live_mdr.news_dataset
            LIMIT 10"""
query_job = client.query(query)
result = query_job.result()
df = result.to_dataframe()
df

,news_id,category_id,title,description,link,image,added_date,source,sub_cat
0,5,1,"Marie Arconati Visconti, une vie à l’École des...","Conférence de Martine Poulain, conservatrice g...",https://www.youtube.com/watch?v=dMuWvA1J29Y,https://www.youtube.com/v/dMuWvA1J29Y?version=3,2024-03-20 09:00:00,École nationale des Chartes : youtube,None
1,158,2,Hangman's Chair - Hellfest 2022 - ARTE Concert,#HANGMANSCHAIR #HELLFEST2022\n\n@hangmanschair...,https://www.youtube.com/watch?v=wsshpam4cE0,https://www.youtube.com/v/wsshpam4cE0?version=3,2024-03-20 09:00:00,Arte : concert : youtube,None
2,682,1,Jakez Karter : Kavadenner ar C'hanada 🗺 /🇨🇦 Ja...,🇨🇦 Nikolaz a lak ar gaoz war Jakez Karter hag ...,https://www.youtube.com/watch?v=q2Rwb-oT5-4,https://www.youtube.com/v/q2Rwb-oT5-4?version=3,2024-03-20 09:00:00,Brezhoweb : youtube,None
3,688,1,Victoria Silvstedt se confie sur Christophe De...,00:00 Intro\n03:45 Son portrait\n23:58 Les cou...,https://www.youtube.com/watch?v=9K5nkRddfIw,https://www.youtube.com/v/9K5nkRddfIw?version=3,2024-03-20 09:00:00,C8 - la chaîne télé : youtube,None
4,709,1,Le Hip-hop doit-il être enseigné comme la dans...,Toutes les émissions de Clique sont à voir gra...,https://www.youtube.com/watch?v=ug71ZAvfz8U,https://www.youtube.com/v/ug71ZAvfz8U?version=3,2024-03-20 09:00:00,Canal Plus : Clique : youtube,None
5,710,1,Une journée avec Hamza dans son quartier - Mod...,Toutes les émissions de Clique sont à voir gra...,https://www.youtube.com/watch?v=Jd5YhQWzDmU,https://www.youtube.com/v/Jd5YhQWzDmU?version=3,2024-03-20 09:00:00,Canal Plus : Clique : youtube,None
6,711,1,"Guillaume Canet : ""Je souhaiterais être plus a...",Toutes les émissions de Clique sont à voir gra...,https://www.youtube.com/watch?v=Mana1X9qIvI,https://www.youtube.com/v/Mana1X9qIvI?version=3,2024-03-20 09:00:00,Canal Plus : Clique : youtube,None
7,712,2,Thylacine : « Faire un concert avec un orchest...,On a suivi le compositeur Thylacine et ses 74 ...,https://www.youtube.com/watch?v=lVw7Nk-5I7U,https://www.youtube.com/v/lVw7Nk-5I7U?version=3,2024-03-20 09:00:00,Canal Plus : musique : youtube,None
8,713,1,Ana Girardot se confie sur le #postpartum dans...,Le Plus de CANAL+ : Ce qu'il ne fallait pas ra...,https://www.youtube.com/watch?v=feJerM2J2IA,https://www.youtube.com/v/feJerM2J2IA?version=3,2024-03-20 09:00:00,Canal Plus : youtube,None
9,731,2,LES PLUS BELLES CHANSONS CORSES CHARLES ROCCHI...,Chansons Corses Charles Rocchi Per Te olivi mu...,https://www.youtube.com/watch?v=JPF9DTLl6VU,https://www.youtube.com/v/JPF9DTLl6VU?version=3,2024-03-20 09:00:00,Chansons Corses Corsica Kallisté Coppelia Oliv...,None


# Implémentation de la TABLE User

In [13]:
username = ['marru', 'dlouvegnies', 'rvincenti']
password = ['marrumdp', 'dlouvegniesmdp', 'rvincentimdp' ]

user_df = pd.DataFrame({'user_id': list(range(1, len(username) + 1)),
        'username': username,
        'password': password})
user_df

,user_id,username,password
0,1,marru,marrumdp
1,2,dlouvegnies,dlouvegniesmdp
2,3,rvincenti,rvincentimdp


In [14]:
# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
USER_TABLE='user_dataset'
USER_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{USER_TABLE}' # the-mdr-project.live_mdr.user_dataset

# Path to your service account key file
key_path = "../credentials/mathieu_credentials.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

In [15]:
client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)
write_mode = 'WRITE_TRUNCATE'
job_config = bigquery.LoadJobConfig(write_disposition=write_mode)
job = client.load_table_from_dataframe(user_df, USER_TABLE_ID, job_config=job_config)
result = job.result()

,user_id,username,password
0,1,marru,marrumdp
1,2,dlouvegnies,dlouvegniesmdp
2,3,rvincenti,rvincentimdp


In [14]:
# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
NEWS_TABLE='news_dataset'
NEWS_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{NEWS_TABLE}' # the-mdr-project.live_mdr.news_dataset

# Path to your service account key file
key_path = "../credentials/mathieu_credentials.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)

query = """SELECT ROW_NUMBER() OVER() AS row_number, news_id, LENGTH(TO_JSON_STRING(t)) AS taille_ligne
            FROM the-mdr-project.live_mdr.news_dataset t
            ORDER BY taille_ligne DESC
            LIMIT 100
        """

job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)

for row in query_job.result():
    print("Index:", row.row_number)
    print("ID de la news:", row.news_id)
    print("Taille de la ligne", row.taille_ligne)

Index: 105243
ID de la news: 270068
Taille de la ligne 427661
Index: 311294
ID de la news: 201838
Taille de la ligne 204703
Index: 132865
ID de la news: 122176
Taille de la ligne 187790
Index: 131989
ID de la news: 65958
Taille de la ligne 162318
Index: 109643
ID de la news: 325401
Taille de la ligne 154278
Index: 133728
ID de la news: 313314
Taille de la ligne 144840
Index: 133817
ID de la news: 313403
Taille de la ligne 136200
Index: 131738
ID de la news: 41625
Taille de la ligne 88998
Index: 132986
ID de la news: 135984
Taille de la ligne 88796
Index: 304049
ID de la news: 38141
Taille de la ligne 82174
Index: 309218
ID de la news: 119368
Taille de la ligne 74606
Index: 301759
ID de la news: 3452
Taille de la ligne 60731
Index: 131843
ID de la news: 53080
Taille de la ligne 60379
Index: 131545
ID de la news: 23696
Taille de la ligne 60249
Index: 100018
ID de la news: 179707
Taille de la ligne 58543
Index: 133956
ID de la news: 313542
Taille de la ligne 55533
Index: 113327
ID de la n

In [20]:
# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
NEWS_TABLE='news_dataset'
NEWS_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{NEWS_TABLE}' # the-mdr-project.live_mdr.news_dataset

# Path to your service account key file
key_path = "../credentials/mathieu_credentials.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)

query = """
    DELETE FROM the-mdr-project.live_mdr.news_dataset
    WHERE LENGTH(TO_JSON_STRING(t)) > 500000
    """

job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)